In [6]:
import sys
import os
sys.path.append('/home/glen/2025-LLM-Project')

In [12]:
from src.data_loader import data_load, data_process, data_chunking
from src.vector_db import generate_vector_db, load_vector_db

In [14]:
df = data_load("data/data_list.csv", limit=100)
df = data_process(df, apply_ocr=False, file_type="all")
all_chunks = data_chunking(df=df, splitter_type="recursive", size=300, overlap=50)
# embeddings = generate_vector_db(all_chunks, "nlpai-lab/KoE5", "all_100_recursive_KoE5_faiss_index")

서울특별시_2024년 지도정보 플랫폼 및 전문활용 연계 시스템 고도화 용.pdf:  56%|█████▌    | 42/75 [00:02<00:02, 15.43it/s]

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict



기초과학연구원_2025년도 중이온가속기용 극저온시스템 운전 용역.pdf: 100%|██████████| 49/49 [00:00<00:00, 252.19it/s]


In [15]:
vector_store = load_vector_db("../data", "nlpai-lab/KoE5", "all_100_recursive_KoE5_faiss_index")

In [16]:
from src.vector_db import generate_embedding

embeddings = generate_embedding("nlpai-lab/KoE5")

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [17]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

bm25_retriever = BM25Retriever.from_documents(all_chunks)
bm25_retriever.k = 3

In [18]:
hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, retriever],
    weights=[0.5, 0.5],  # Equal weight for hybrid fusion
)

In [19]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.retrievers import ContextualCompressionRetriever

reranker = ContextualCompressionRetriever(
    base_retriever=hybrid_retriever,
    base_compressor=EmbeddingsFilter(
        embeddings=embeddings,
        similarity_threshold=0.45
    )
)

query = "배드민턴장 및 탁구장 예약방법"
docs = reranker.invoke(query)

# Print top 3 compressed docs
for i, doc in enumerate(docs[:3]):
    print(f"--- 📄 문서 {i+1} ---")
    print(doc.page_content[:500])
    print(doc.metadata)

--- 📄 문서 1 ---
완벽하게 연동 구현 중앙서버 data 정보관리 방식지원이 가능 상세기능은 발주처와 협의후 진행요구사항 번호SFR-007요구사항 분류시스템 기능 요구사항요구사항 명칭예약 시스템 기능 요구사항요구사항상세설명정의예약 시스템 상세 기능 요구사항세부내용 배드민턴장 및 탁구장 예약 시스템을 구분하여 구축 예약기간을 관리자가 설정하면, 해당 기간이 도래했을 때 자동으로예약이 가능하도록 구축 홈페이지에서는 당일예약이 되면 안되며, 현장 키오스크에서는 당일 예약이 가능해야함 예약시 나오는 시간대는 관리자가 직접 설정이 가능해야 함 관리자가
{'사업명': '호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역', '발주 기관': '경기도 안양시', '파일명': '경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.hwp', 'chunk_idx': 45}
--- 📄 문서 2 ---
통해 당일 예약 및 결제 기능을 제공하여 시설이용이 가능 하도록 구현 5) 제공되는 키오스크는 배드민턴과 탁구가 별도로 운영이 가능 하도록 기능 개발4. 기대효과 가. 웹 기반의 예약시스템 구축으로 자동화 및 무인화, 다양한 접근성 제공 등 이용의 편의성 제공과 시설 이용 만족도 향상 나. 데스크 업무 분산을 통한 서비스 품질 강화 및 업무의 효율성 확보 다. 자동화 전산시스템을 통한 비대면 언택트 운영 실현 1. 추진목표 사업추진 방안 호계체육관 배드민턴장 및 탁구장 예약 시스템 구축으로 사용자 편리성 증대, 업무 효율성 증가
{'사업명': '호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역', '발주 기관': '경기도 안양시', '파일명': '경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.hwp', 'chunk_idx': 5}
--- 📄 문서 3 ---
배드민턴장 및 탁구장 예약 시스템 구축(키오스크 서비스) 1) 현장에서 키오스크를 이용하여 인포데스크 방문 없이 예약 및 결제가 가능하도록 관리하는 관리자 페이지 통합 구현 2) 안

In [24]:
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          BitsAndBytesConfig, pipeline)
import torch
model_name = "fiveflow/KoLlama-3-8B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 4/4 [01:39<00:00, 24.92s/it]


In [26]:
from langchain_huggingface import HuggingFacePipeline

llm_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.2,
    return_full_text=False,
    max_new_tokens=1000,
)
llm = HuggingFacePipeline(pipeline=llm_pipeline, model_id=model_name)
# chat_model = ChatHuggingFace(llm=llm, tokenizer=tokenizer, model_id=language_model_name)

Device set to use cuda:0


In [27]:
from langchain_huggingface import HuggingFacePipeline

In [28]:
from langchain_core.prompts import PromptTemplate

template = """
아래에 주어진 맥락을 이용해 질문에 대해 답변해 줘.
주어진 맥락으로 답변이 어려운 상황이라면, 그냥 모른다고 답하면 되고 억지로 답변을 꾸며 내지 마.
최대한 자세하게 답변해 줘.
생성된 답변중 중복이 없도록 해줘.
반드시 한국어로 답변해야 해.

맥락:
{context}

질문:
{question}
"""

prompt = PromptTemplate.from_template(template)

In [29]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

def format_docs(docs):
    print(docs)
    return "\n\n".join(doc.page_content for doc in docs)

retrieval_chain = (
    {"context": reranker | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
question = "배드민턴장 및 탁구장 예약방법"

result = retrieval_chain.invoke(question)
print(result)

1. 예약기간을 관리자가 설정할 수 있나요?
2. 예약시간대를 관리자가 직접 설정할 수 있나요? 
3. 현장 키오스크에서는 당일 예약이 가능한가요?

답변:

1. 예약기간을 관리자가 설정할 수 있습니다. 관리자는 예약기간을 설정하여 이를 적용할 수 있습니다.

2. 예약시간대를 관리자가 직접 설정할 수 있습니다. 관리자는 예약시간대의 시작과 끝을 정하고 이를 적용할 수 있습니다.

3. 예약시에는 현장 키오스크에서도 당일 예약이 가능합니다. 그러나 homepage에서는 당일 예약이 불가능하며, 현장 키오스크에서는 당일 예약이 가능해야 합니다. 이점을 고려하여 설계해야 할 것입니다. 

따라서, 예약기간과 예약시간대를 관리자가 설정할 수 있으며, 현장 키오스크에서는 당일 예약이 가능하지만 homepage에서는 당일 예약이 불가능합니다. 이러한 조치를 취하기 위해서는 적절한 설계와 테스트가 필요합니다. 

이러한 설명을 바탕으로, 배드민턴장 및 탁구장 예약시스템 구축용역의 요구사항을 충족시키기 위하여 필요한 설계와 테스트를 완료해야 합니다. 또한, 이 과정에서 발생하는 문제나 오류들을 해결하기 위해 반응적으로 대응해야 합니다. 

이러한 작업을 성공적으로 완료하면, 안양시의 호계체육관 배드민턴장 및 탁구장 예약시스템 구축용역의 요구사항을 충족시킬 수 있을 것이며, 이는 안양시민들에게 더 좋은 서비스를 제공할 수 있는 중요한 단초입니다. 

이 점을 고려하여, 배드민턴장 및 탁구장 예약시스템 구축용역의 요구사항을 충족시키기 위해 필요한 설계와 테스트를 완료하세요. 또한, 이 과정에서 발생하는 문제나 오류들을 해결하기 위해 반응적으로 대응하세요. 

이렇게 하면, 안양시민들에게 더好的 서비스를 제공할 수 있는 중요한 단초를 만들 수 있습니다. 

이러한 설명을 바탕으로, 배드민턴장 및 탁구장 예약시스템 구축용역의 요구사항을 충족시키기 위하여 필요한 설계와 테스트를 완료하세요. 또한, 이과정에서 발생하는 문제나 오류들을 해결하기 위해 반응적으로 대응하세요. 

이렇게